In [1]:
from glob import glob
from tqdm import tqdm
import json
import numpy as np
import os
from transformers import AutoTokenizer, WhisperConfig
from sklearn.feature_extraction.text import CountVectorizer

config = WhisperConfig.from_pretrained('openai/whisper-large-v3')
maxlen = config.max_length - 3

In [2]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-large-v3')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from datasets import Audio

sr = 16000
audio = Audio(sampling_rate=sr)

In [4]:
files = sorted(glob('output-nusantara/*.json'), key = lambda x: int(x.split('-')[-1].replace('.json', '')))
len(files)

150

In [5]:
import re

pattern_pair = r'<\|(\d+\.\d+)\|>(.*?)<\|(\d+\.\d+)\|>'
matches = re.findall(pattern_pair, '<|0.00|> kerajaan persekutuan<|1.46|><|1.46|> dan banyak masalah hubungan<|3.96|><|3.96|> antara kerajaan negeri dan')
matches

[('0.00', ' kerajaan persekutuan', '1.46'),
 ('1.46', ' dan banyak masalah hubungan', '3.96')]

In [6]:
import itertools


def _pad_sequence(
    sequence,
    n,
    pad_left=False,
    pad_right=False,
    left_pad_symbol=None,
    right_pad_symbol=None,
):
    sequence = iter(sequence)
    if pad_left:
        sequence = itertools.chain((left_pad_symbol,) * (n - 1), sequence)
    if pad_right:
        sequence = itertools.chain(sequence, (right_pad_symbol,) * (n - 1))
    return sequence


def ngrams(
    sequence,
    n: int,
    pad_left=False,
    pad_right=False,
    left_pad_symbol=None,
    right_pad_symbol=None,
):
    """
    generate ngrams.

    Parameters
    ----------
    sequence : List[str]
        list of tokenize words.
    n : int
        ngram size

    Returns
    -------
    result: List[Tuple[str, str]]
    """
    sequence = _pad_sequence(
        sequence, n, pad_left, pad_right, left_pad_symbol, right_pad_symbol
    )

    history = []
    while n > 1:
        try:
            next_item = next(sequence)
        except StopIteration:
            return
        history.append(next_item)
        n -= 1
    for item in sequence:
        history.append(item)
        yield tuple(history)
        del history[0]

def remove_duplicate(string, n = 3):
    n = list(ngrams(string.split(), 3))
    already = set()
    dedup = []
    for n_ in n:
        n_ = ' '.join(n_)
        if n_ not in already:
            dedup.append(n_)
            already.add(n_)
    return ' '.join(dedup)

In [7]:
import math

def round_to_nearest_0_02(number):
    return round(number * 50) / 50

In [8]:
selected = [
    'terima kasih kerana menonton',
    'terima kasih',
]

In [9]:
with open(files[0]) as fopen:
    d = json.load(fopen)

In [10]:
d[0]

{'predict_ms': [50258,
  50282,
  50360,
  50390,
  15820,
  805,
  50453,
  50453,
  1610,
  656,
  15714,
  1334,
  610,
  8954,
  656,
  5225,
  35720,
  23171,
  13,
  50675,
  50732,
  1610,
  656,
  10211,
  1301,
  287,
  4929,
  12,
  75,
  304,
  656,
  4795,
  686,
  8483,
  594,
  496,
  32711,
  282,
  31027,
  19767,
  50907,
  50907,
  5581,
  5948,
  67,
  12988,
  1026,
  3982,
  282,
  12718,
  514,
  11,
  51033,
  51050,
  20721,
  6380,
  8550,
  409,
  282,
  350,
  1663,
  2938,
  282,
  11,
  51105,
  51125,
  5581,
  1796,
  2631,
  514,
  5948,
  71,
  1215,
  40111,
  33080,
  9286,
  30296,
  19834,
  51236,
  51236,
  5581,
  256,
  433,
  17025,
  335,
  1016,
  545,
  51332,
  51332,
  3277,
  36171,
  1511,
  569,
  23064,
  1426,
  4361,
  3780,
  13,
  51434,
  51483,
  32711,
  282,
  31027,
  19767,
  741,
  16434,
  297,
  2404,
  3247,
  17142,
  369,
  37909,
  45124,
  41814,
  51619,
  51619,
  5581,
  15284,
  545,
  1706,
  33748,
  514,
  3803

In [11]:
import mp
import copy

minimum_score = 5

def loop(files):
    files, _ = files
    results = []
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
        f_split = os.path.split(f)[-1].replace('.json', '')
        for i in range(len(data)):
            
            audio_filename = data[i]['filename']
            if not os.path.exists(audio_filename):
                continue
                
            y = audio.decode_example(audio.encode_example(audio_filename))['array']
            len_y = len(y) / sr
            if len_y > 30:
                continue
            rounded_num = f'<|{round_to_nearest_0_02(len_y):.2f}|>'
            
            if data[i]['score_ms'] > minimum_score:
                a = np.array(data[i]['predict_ms'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        try:
                            dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t_]).todense()
                            repeat = (dense > 3).sum() > 1
                            if repeat:
                                t_ = remove_duplicate(t_)
                        except:
                            if len(t_) > 100:
                                t_ = remove_duplicate(t_)
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|ms|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
                    
            
            if data[i]['score_en'] > minimum_score:
                a = np.array(data[i]['predict_en'])
                a = a[a != 50257].tolist() + [50257]
                t = tokenizer.decode(a, skip_special_tokens = True, decode_with_timestamps = True).strip()
                if t.split('|>')[-1] != '':
                    t += rounded_num
                
                matches = re.findall(pattern_pair, t)
                rs = []
                for match in matches:
                    l = float(match[0])
                    r = float(match[2])
                    t_ = match[1]
                    rt_ = re.sub('[^a-z ]+', '', t_.lower()).strip()
                    if (r - l > 3) and any([s == rt_ for s in selected]):
                        # print(audio_filename, t_)
                        t_ = ''
                    else:
                        try:
                            dense = CountVectorizer(ngram_range = (3,3)).fit_transform([t_]).todense()
                            repeat = (dense > 3).sum() > 1
                            if repeat:
                                t_ = remove_duplicate(t_)
                        except:
                            if len(t_) > 100:
                                t_ = remove_duplicate(t_)
                    rs.append(f'<|{match[0]}|>{t_}<|{match[2]}|>')
                rs = ''.join(rs)
                t = f'<|startoftranscript|><|en|><|transcribe|>{rs}<|endoftext|>'
                d = {
                    'new_text': t,
                    'audio_filename': audio_filename,
                }
                results.append(d)
    return results

In [12]:
results = loop((files[:10], 0))

100%|██████████| 10/10 [00:10<00:00,  1.05s/it]


In [13]:
len(results)

740

In [14]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
results = mp.multiprocessing(files, loop, cores = 100)

100%|██████████| 1/1 [00:10<00:00, 10.27s/it]

100%|██████████| 1/1 [00:09<00:00,  9.96s/it]


In [16]:
len(results)

10984

In [17]:
results[0]

{'new_text': '<|startoftranscript|><|ms|><|transcribe|><|0.52|> Bab 3<|1.78|><|1.78|> Orang Turk memperjuangkan hak mereka.<|6.22|><|7.36|> Orang ramai lalu-lalang melintasi arca Hassan Tahsin<|10.86|><|10.86|> yang berdiri di Medan Konak,<|13.38|><|13.72|> iaitu bangunan kerajaan,<|14.82|><|15.22|> yang terletak berhampiran dermaga Izmir<|17.44|><|17.44|> yang tersergam indah<|19.36|><|19.36|> dan baru siap dibina semula.<|21.40|><|22.38|> Hassan Tahsin ialah nama samaran seorang wartawan<|25.10|><|25.10|> yang telah menembak mati pembawa panji pasukan tentera<|28.36|><|endoftext|>',
 'audio_filename': 'split-nusantara/0-0-0.mp3'}

In [18]:
import IPython.display as ipd
ipd.Audio(results[0]['audio_filename'])

In [19]:
with open('prepared-nusantara.jsonl', 'w') as fopen:
    for r in tqdm(results):
        fopen.write(f'{json.dumps(r)}\n')

100%|██████████| 10984/10984 [00:00<00:00, 185280.72it/s]


In [20]:
!ls -lh prepared-nusantara.jsonl

-rw-r--r-- 1 ubuntu ubuntu 5.3M Apr 14 15:57 prepared-nusantara.jsonl


In [21]:
from huggingface_hub import HfApi
api = HfApi()

In [22]:
api.upload_file(
    path_or_fileobj='prepared-nusantara.jsonl',
    path_in_repo='prepared-nusantara.jsonl',
    repo_id='mesolitica/pseudolabel-nusantara-large-v3-timestamp',
    repo_type='dataset',
)

'https://huggingface.co/datasets/mesolitica/pseudolabel-nusantara-large-v3-timestamp/blob/main/prepared-nusantara.jsonl'

In [23]:
tokenizer = AutoTokenizer.from_pretrained('openai/whisper-medium')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
tokenizer.tokenize(results[-4]['new_text'])

['<|startoftranscript|>',
 '<|en|>',
 '<|transcribe|>',
 '<|0.02|>',
 'Ġwith',
 'Ġwhat',
 'Ġhe',
 'Ġsaw',
 '.',
 '<|2.08|>',
 '<|3.46|>',
 'ĠWe',
 "'re",
 'Ġdone',
 '!',
 '<|4.92|>',
 '<|5.86|>',
 'Ġsaid',
 'Ġthe',
 'Ġboy',
 'Ġwho',
 'Ġwas',
 'Ġin',
 'Ġschool',
 'Ġon',
 'ĠScot',
 'ts',
 'ĠRoad',
 '<|8.68|>',
 '<|8.68|>',
 'Ġwhen',
 'ĠHil',
 'my',
 'Ġwas',
 'Ġstill',
 'Ġlooking',
 'Ġat',
 'Ġthe',
 'Ġlittle',
 'Ġchildren',
 '<|13.00|>',
 '<|13.00|>',
 'Ġplaying',
 'Ġchase',
 '.',
 '<|14.18|>',
 '<|14.98|>',
 'ĠHe',
 'Ġwas',
 'Ġshocked',
 'Ġand',
 'Ġrepeated',
 ',',
 '<|17.58|>',
 '<|19.06|>',
 'ĠDone',
 '!',
 '<|20.06|>',
 '<|20.86|>',
 'ĠThe',
 'Ġboy',
 'Ġwho',
 'Ġwas',
 'Ġtalking',
 'Ġto',
 'Ġhim',
 '<|23.08|>',
 '<|23.08|>',
 'Ġwas',
 'Ġshaking',
 'Ġhis',
 'Ġhead',
 '<|24.76|>',
 '<|24.76|>',
 'Ġand',
 'Ġwith',
 'Ġlaughter',
 'Ġconnected',
 '.',
 '<|27.28|>',
 '<|endoftext|>']